In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_train = pd.read_csv('f:/github/tusonggao/Kaggle_code/titanic/data_bakup/train.csv')
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [3]:
data_train.groupby('Survived').count()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Survived,,,,,,,,,,,
0,549,549,549,549,424,549,549,549,549,68,549
1,342,342,342,342,290,342,342,342,342,136,340


In [4]:
data_train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [5]:
def set_missing_ages(p_df):
    p_df.loc[(p_df.Age.isnull()), 'Age'] = p_df.Age.dropna().mean()
    return p_df
df = set_missing_ages(data_train)

In [6]:
import sklearn.preprocessing as preprocessing
scaler = preprocessing.StandardScaler()
print(np.sum(data_train['Age'].isnull()))
df['Age_scaled'] = scaler.fit_transform(data_train['Age'])
df['Fare_scaled'] = scaler.fit_transform(data_train['Fare'])

0


c:\Program Files\Anaconda3_3.5\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
c:\Program Files\Anaconda3_3.5\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
c:\Program Files\Anaconda3_3.5\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your dat

In [7]:
def set_cabin_type(p_df):
    p_df.loc[(p_df.Cabin.notnull()), 'Cabin'] = 'Yes'
    p_df.loc[(p_df.Cabin.isnull()), 'Cabin'] = 'No'
    return p_df
df = set_cabin_type(data_train)

In [8]:
dummies_pclass = pd.get_dummies(data_train['Pclass'], prefix='Pclass')
dummies_pclass.head(3)

,Pclass_1,Pclass_2,Pclass_3
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0


In [9]:
dummies_embarked = pd.get_dummies(data_train['Embarked'], prefix='Emarked')
dummies_embarked.loc[61]

Emarked_C    0.0
Emarked_Q    0.0
Emarked_S    0.0
Name: 61, dtype: float64

In [10]:
dummies_sex = pd.get_dummies(data_train['Sex'], prefix='Sex')
dummies_sex.head(3)

,Sex_female,Sex_male
0,0.0,1.0
1,1.0,0.0
2,1.0,0.0


In [11]:
df = pd.concat([df, dummies_embarked, dummies_sex, dummies_pclass], axis=1)
df.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)
train_df = df.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*')
train_df.head(1)

,Survived,SibSp,Parch,Age_scaled,Fare_scaled,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
0,0,1,0,-0.592481,-0.502445,0.0,1.0,0.0,0.0,1.0


In [12]:
from abupy import AbuML

train_np = train_df.as_matrix()
y = train_np[:, 0]
x = train_np[:, 1:]
titanic = AbuML(x, y, train_df)

titanic.estimator.logistic_classifier()
titanic.cross_val_accuracy_score()

LogisticRegression score mean: 0.7901574736125297


array([ 0.7889,  0.7778,  0.764 ,  0.809 ,  0.7978,  0.764 ,  0.7865,
        0.7753,  0.8315,  0.8068])

In [13]:
df['Child'] = (data_train['Age'] <= 10).astype(int)
df['Age*Age'] = data_train['Age'] * data_train['Age']
df['Age*Age_scaled'] = scaler.fit_transform(df['Age*Age'])

In [14]:
df['Age*Class'] = data_train['Age'] * data_train['Pclass']
df['Age*Class_scaled'] = scaler.fit_transform(df['Age*Class'])
train_df = df.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*|Child|Age\*Class_.*')
train_df.head(1)

,Survived,SibSp,Parch,Age_scaled,Fare_scaled,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Child,Age*Age_scaled,Age*Class_scaled
0,0,1,0,-0.5925,-0.5024,0.0,1.0,0.0,0.0,1.0,0,-0.6366,0.0314


In [15]:
train_np = train_df.as_matrix()
y = train_np[:, 0]
x = train_np[:, 1:]
titanic = AbuML(x, y, train_df)

titanic.estimator.logistic_classifier()
titanic.cross_val_accuracy_score()

LogisticRegression score mean: 0.8069992055385313


array([ 0.8222,  0.7667,  0.7865,  0.8427,  0.8202,  0.7753,  0.7865,
        0.7865,  0.8652,  0.8182])

In [16]:
titanic.importances_coef_pd()

,coef,columns
0,[-0.435531044842],SibSp
1,[-0.189514179528],Parch
2,[-0.201623667229],Age_scaled
3,[0.198925435282],Fare_scaled
4,[1.68056556924],Sex_female
5,[-1.07528904566],Sex_male
6,[0.731262670617],Pclass_1
7,[0.0],Pclass_2
8,[-0.925327778345],Pclass_3
9,[1.27647191133],Child


In [17]:
titanic.feature_selection()

RFE selection
                  ranking support
SibSp                   1    True
Parch                   3   False
Age_scaled              2   False
Fare_scaled             4   False
Sex_female              1    True
Sex_male                1    True
Pclass_1                1    True
Pclass_2                7   False
Pclass_3                1    True
Child                   1    True
Age*Age_scaled          6   False
Age*Class_scaled        5   False


,ranking,support
SibSp,1,True
Parch,3,False
Age_scaled,2,False
Fare_scaled,4,False
Sex_female,1,True
Sex_male,1,True
Pclass_1,1,True
Pclass_2,7,False
Pclass_3,1,True
Child,1,True
